In [7]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
import numpy as np
from tqdm import tqdm
import os
import sys

In [2]:
data_path_human = "../data/rDany/human_text.txt"
data_path_robot = "../data/rDany/robot_text.txt"

with open(data_path_human, "r") as f:
    human_lines = f.read().split("\n")
    
with open(data_path_robot, "r") as f:
    robot_lines = f.read().split("\n")
print(human_lines[1])
print(robot_lines[1])

oh, thanks !  i'm fine. this is an evening in my timezone
😄 here is afternoon ! 


In [36]:
vocabulary_size = 10000
sentence_size = 258

text_vectorization_layer = TextVectorization(max_tokens=vocabulary_size, output_sequence_length=sentence_size)
text_vectorization_layer.adapt(human_lines + robot_lines)

In [8]:
with open("../data/glove.twitter.27B.50d.txt", "r") as f:
    dict_w2v = {}
    problems = []
    
    for line in tqdm(f):
        
        tokens = line.split()
        
        word = tokens[0]
        vector = np.array(tokens[1:], dtype=np.float32)
        
        if vector.shape[0] == 50:
            dict_w2v[word] = vector
        else:
            problems.append({word: vector})

1193514it [00:27, 44070.25it/s]


In [17]:
num_tokens = len(text_vectorization_layer.get_vocabulary())
embedding_dim = 50
hits = 0
misses = 0
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for token in tqdm(text_vectorization_layer.get_vocabulary()):
    
    embedding = dict_w2v.get(token)
    if embedding is not None:
        embedding_matrix[text_vectorization_layer([token]).numpy()[0, 0]] = embedding
        hits += 1
    else:
        misses += 1
    

100%|██████████| 5485/5485 [00:09<00:00, 565.85it/s]


In [22]:
embedding_matrix[2]

array([ 5.10279983e-02,  4.86490011e-01, -1.83469996e-01,  3.30749989e-01,
       -4.70070004e-01, -6.94120005e-02,  1.13100004e+00,  3.17909986e-01,
       -1.24720000e-01,  8.28909993e-01, -3.50919992e-01,  5.10840006e-02,
       -6.40070009e+00, -8.04430008e-01, -8.51570010e-01,  2.32920006e-01,
       -8.70200023e-02, -4.23040003e-01, -2.67019987e-01, -3.67500004e-03,
        4.65970010e-01,  1.35539994e-01,  1.04249999e-01,  9.52400029e-01,
       -4.85289991e-02,  2.45450005e-01,  3.84009987e-01,  2.28350004e-03,
        1.19099998e+00, -3.03739995e-01, -4.44409996e-01,  8.26079994e-02,
        2.38509998e-01,  3.32679987e-01,  1.32699996e-01,  3.87419999e-01,
        5.44470012e-01,  2.11710006e-01, -8.30610022e-02,  2.97399998e-01,
       -1.08580005e+00, -3.62289995e-01,  4.38259989e-01,  3.00089985e-01,
       -7.43770003e-02, -3.87529999e-01, -2.26109996e-01,  7.15169981e-02,
        1.34729996e-01, -7.37999976e-02])

In [39]:
model = tf.keras.models.Sequential()

model.add(text_vectorization_layer)
model.add(tf.keras.layers.Embedding(
    num_tokens,
    embedding_dim,
    tf.keras.initializers.Constant(embedding_matrix),
    trainable=True))
model.add(tf.keras.layers.LSTM(128))
model.add(tf.keras.layers.Dense(10))
model.compile(optimizer='adam',
              loss='BinaryCrossentropy',
              metrics=['accuracy', 'Precision'])

In [44]:
model(["cat is there on the window"])

Consider rewriting this model with the Functional API.


<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[-2.4095742e-20, -8.4734786e-20,  9.8878463e-20, -8.4335270e-20,
        -8.8498031e-20,  1.1737989e-19,  2.4859112e-21, -3.1846722e-19,
         1.6712448e-19, -2.2653656e-20]], dtype=float32)>